# Interatomic force field (IFF) optimization for W-based metal alloys

Author: Lukas Vlcek

Start Date: 2018-04-22

In [1]:
from datetime import datetime ; print('Last update:', datetime.now())

Last update: 2018-07-25 21:53:17.329834


# Table of Contents

* [1. Introduction](#1.-Introduction)
* [2. Theoretical background](#2.-Theoretical-background)
    * [2.1 Model definition](#2.1-Model-definition)
    * [2.2 Optimization approach](#2.2-Optimization-approach)
        * [2.2.1 Perturbation technique](#2.2.2-Perturbation-technique)
        * [2.2.2 Statistical distance loss function](#2.2.1-Statistical-distance-loss-function)
    * [2.3 Target data](#2.3-Target-data)
    * [2.4 Simulation details](#2.4-Simulation-details)
* [3. Optimization](#3.-Optimization)

**Notebook setup**

In [37]:
# Import libraries
%matplotlib inline
import os
import glob
import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import h5py
from itertools import product
from scipy.optimize import fmin

# Paths to important directories
pot_path = '../sim/potentials'
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

## 1. Introduction

**Goal:** Optimize EAM potential for W using the functional form of Bonny et al. (2017), and target data from Marinica et al. (2013) and German. Show that using this simplified EAM form we can develop a more predictive model of W compared to EAM2 model of Marinica.

## 2. Theoretical background

### 2.1 Model definition

**Equilibrium potential**

Energy of an N-particle configuration

$$ E = \sum_{i=1}^N \left[ \sum_{j>i}^N V_{t_it_j}\left(r_{ij}\right) + F_{t_i}\left(\rho_i\right) \right] $$

Here $V_{t_it_j}$ is pair interaction between atom types $t_i$ and $t_j$ at distance $r_{ij}$ defined as

$$ V_{t_it_j}\left(r_{ij}\right) = \sum_{k=1}^{N_p}\left[a_k\left(r_k - r_{ij}\right)^3\Theta\left(r_k - r_{ij}\right)\right]\ $$

where $\Theta$ is Heaviside step function.
$F_{t_i}$ is the manybody embedding function

$$ F_{t_i}(\rho_i) = A_{t_i}\sqrt{\rho_i} + B_{t_i}\rho + C_{t_i}\rho^2 $$

where $\rho_i$ effective electron density 

$$ \rho_i = \sum_{j\ne i}^N \phi_{t_j}\left(r_{ij}\right) $$

and $\phi$ is cohesive potential

$$ \phi\left(r_{ij}\right) = D_{t_j}\left(r_c - r_{ij}\right)^3\Theta\left(r_c - r_{ij}\right) $$

In [3]:
# Define the equilibrium EAM functions (to be optimized)

# Pair potential (cubic splines). Parameters: distance (r), spline parameters (aa), spline nodes (cc)
V = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

# Embedding function. Parameters: electronic density (d), coefficients for 1/2, 1, and 2 powers of density
F = lambda d, a: a[0]*d**0.5 + a[1]*d + a[2]*d**2

# Cohesive potential (cubic splines - same form as V)
phi = V

**Core and transition potential for short and intermediate distances**

In [4]:
# Define the core and transition parts of the potential (kept constant)
def u_core(r, za=74, zb=74):
    """Repulsive potential of the atomic cores. Default atomic numbers for W"""
    qe_sq = 14.3992 # squared electron charge  
    rs = 0.4683766/(za**(2/3) + zb**(2/3))**0.5
    x = r/rs
    u  = 0.1818*np.exp(-3.2*x)
    u += 0.5099*np.exp(-0.9423*x)
    u += 0.2802*np.exp(-0.4029*x)
    u += 0.02817*np.exp(-0.2016*x)
    u *= za*zb*qe_sq/r
    return u

def u_trans(r, unucl, ueq, ri=1.0, ro=2.0):
    """Transition between the core and equillibrium potentials"""
    if r < ri:
        return unucl
    elif r < ro:
        x = (ro + ri - 2*r)/(ro - ri)
        eta = 3/16*x**5 - 5/8*x**3 + 15/16*x + 1/2
        return ueq + eta*(unucl - ueq)
    else:
        return ueq    

### 2.2 Optimization approach

#### 2.2.1 Perturbation technique

**Pair distances**

In [6]:
def pair_dist(xyz, box):
    """
    Calculates nearest image pair distances between all atoms in xyz array.
    Parameters
    -----------
    xyz : numpy array
          particle x, y, z coordinates
    box : scalar or numpy array
          simulation box dimensions/shape
    Returns
    -------
    rr  : (natom, natom) numpy array of pair distances
    rx  : (natom, natom, 3) numpy array of pair distance coordinates
    """

    n_atom = xyz.shape[0] # number of atoms in a configuration
    rr = np.empty((n_atom, n_atom), dtype=float)
    rx = np.empty((n_atom, n_atom, 3), dtype=float)

    for i, pa in enumerate(xyz):
        for j, pb in enumerate(xyz):
            dp = pa - pb
            dp = np.where(dp < -0.5*box, dp + box, dp)
            dp = np.where(dp >  0.5*box, dp - box, dp)
            rr[i,j] = np.sum(dp*dp)**0.5
            rx[i,j] = dp

    return rr, rx

**EAM sufficient statistics**

In [6]:
def get_stats_EAM(rr, rx, sc):
    """
    Takes atom pair distances and calculates sufficeint statistics needed
    for the parameterization of a cubic spline-based EAM model by Bonny et al. (2017).
    
    Parameters
    ----------
    rr : numpy array
         set of pair distances
    rx : numpy array
         set of pair distance coordinates
    sc : python list
         spline nodes
         
    Returns
    -------
    ar, a1, a2 : numpy arrays (len(sc))
                 atom energy-related statistics
                 el_density**0.5, el_density, el_density**2
    br, b1, b2 : numpy arrays (len(sc), natoms, 3 coordinates)
                 atom force-related statistics (gradients of energy)
                 grad(el_density**0.5), grad(el_density), grad(el_density**2)
    """
    
    # number of atoms in configuration
    n_atom = rr.shape[0]
    
    # energy-related statistics
    aa = np.empty((n_atom), dtype=float)
    ar = np.zeros((len(sc)), dtype=float)
    a1 = np.zeros_like(ar)
    a2 = np.zeros_like(ar)
    
    # force-related statistics
    br = np.zeros((len(sc), n_atom, 3), dtype=float)
    b1 = np.zeros_like(br)
    b2 = np.zeros_like(br)
    zero3 = np.zeros((3), dtype=float)

    # cycle over spline nodes
    for ks, rc in enumerate(sc):
        
        # cycle over atoms
        for i in range(n_atom):
            
            # sum electronic density over all neighbors of i within rc
            aa[i] = sum([(rc - r)**3 for r in rr[i] if (r < rc and r > 0.01)])

            # if el. density larger than zero, calculate force statistics
            if aa[i] > 0.0:
                
                # precompute a list of recurring values for force statistics
                ff = [1.5*(rc - r)**2*x/r if (r > 0.01 and r < rc) else zero3 for r, x in zip(rr[i], rx[i])]
                
                # sum contributions to force statistics from all neighbors of i
                b1[ks, i] = sum([2*f       for f in ff])
                br[ks, i] = sum([ -f/np.sqrt(aa[i]) for f in ff])
                b2[ks, i] = sum([4*f*aa[i] for f in ff])

        # sum contributions to energy statistics for a given spline node
        ar[ks] = np.sum(np.sqrt(aa))
        a1[ks] = np.sum(aa)
        a2[ks] = np.sum(aa**2)
        
    return a1, ar, a2, b1, br, b2

**EAM configurational energy based on sufficient statistics and model parameters**

In [16]:
def utot_EAM(params, ustats):
    """
    Calculates configurational energy from EAM sufficient statistics and model parameters

    Parameters
    ----------
    params : list of lists and floats
             EAM interaction parameters (spline coefficients array and embedding function parameters)
    ustats : list of lists and floats
             Sufficient statistics for a trajectory of configurations

    Returns
    -------
    u_total: float
             total configurational energy (sum of pair and manybody interactions) for trajectory of configurations
    """

    n_sample = stats.shape[0]

    # pair interactions from array of spline coefficeints and corresponding statistic
    u_pair = np.array([sum([a*s for a, s in zip(params[2:], ustats[i, 2])]) for i in range(n_sample)])

    # manybody interactions from embedding function parameters and corresponding statistics
    u_many = np.array([params[0]*ustats[i, 0] + params[1]*ustats[i, 1] for i in range(n_sample)])

    u_total = u_pair + u_many

    return u_total

In [17]:
def ftot_EAM(params, fstats):
    """
    Calculates configurational energy from EAM sufficient statistics and model parameters

    Parameters
    ----------
    params : list of lists and floats
             EAM interaction parameters (spline coefficients array and embedding function parameters)
    fstats : list of lists and floats
             Sufficient statistics

    Returns
    -------
    f_total: float
             total configurational energy (sum of pair and manybody interactions)
    """

    # number of samples and atoms
    n_sample = stats.shape[0]
    n_atom = stats.shape[1]

    f_total = np.zeros((n_sample, 6*n_atom + 1), dtype=float)

    # cycle over samples
    for i in range(n_sample):

        # pair interactions from array of spline coefficeints and corresponding statistic
        f_pair = sum([p*s for p, s in zip(params[2:], fstats[i,2])]) 

        # manybody interactions from embedding function parameters and corresponding statistics
        f_many = params[0]*fstats[i,0] + params[1]*fstats[i,1]

        # Create a 6N + 1 array of 0, f, and -f
        f_total[1:3*n_atom+1] = f_pair.flatten() + f_many.flatten()
        f_total[3*n_atom+1:] = -f_total[1:3*n_atom+1]
            
    return f_total

#### 2.2.2 Statistical distance loss function

**Loss function based on configurational energies and forces**

In [18]:
def sd2_loss(params, stats, targets, weights=None):
    """
    Calculates squared statistical distance loss function for configurational energies and forces.

    Parameters
    ----------
    params : list of lists and floats
             EAM interaction parameters (spline coefficients array and embedding function parameters)
    stats  : list of lists and floats
             Sufficient statistics
    targets: list of lists and floats
             target energies and forces
    weights: weighting of different target data

    Returns
    -------
    sd2, sd2f: float
               squared statistical distances between model and target (energy and force-based)
    """

    # apply bounds on parametes
    #p = np.where(p < -1.0, -1.0, p)
    #p = np.where(p >  1.0,  1.0, p)

    # if weights not provided, assign equal weights
    if not weights:
        weights = np.ones((len(targ)), dtype=float)

    # cycle over target system trajectories and statistics to determine SD
    sd2 = sd2f = 0.0
    for targ, stat, w in zip(targets, stats, weights):

        beta = targ['beta'] # system inverse temperature
        u_targ = targ['energy'] # target energies
        u_stat = stat['energy'] # energy statistics
        u_pars = params
        n_sample = u_targ.shape[0]

        # energy diference array for a given target trajectory
        uuu = beta*(utot_EAM(u_pars, u_stat) - u_targ) # array(n_sample)
        uuu -= np.mean(uuu)
        eee = np.exp(-uuu)
 
        # are forces available?
        if 'forces' not in targ:

            # energy-based free energy difference and statistical distance
            ge = -np.log(np.mean(eee))   # free energy difference (shifted)
            cb = np.mean(np.exp(-0.5*(uuu - ge))) # Bhattacharyya coefficient
            sd2 += w*np.arccos(cb)**2              # statistical distance

        else:

            betad = beta*0.01  # beta * dl
            f_targ = targ['forces'] # target forces (n_sample, 1+6N) (0, 3Nf, -3Nf)
            f_stat = stat['forces'] # force statistics (n_sample, npars, 3N)
            f_pars = params

            eeh = np.exp(-0.5*uuu)
            fff = ftot_EAM(f_pars, f_stat) # n_sample *(6N + 1) force contributions

            # target and model force terms
            fpave = np.mean(np.exp(betad*f_targ))
            fqave = np.mean([eee[i]*np.mean(np.exp(betad*fff[i])) for i in range(n_sample)])
            fhave = np.mean([eeh[i]*np.mean(np.exp(0.5*betad*(fff[i]+f_targ[i]))) for i in range(n_sample)])
            
            # force-based free energy difference and statistical distance
            gef = -np.log(fqave/fpave)
            cb = fhave/(fqave*fpave)**0.5
            if cb > 1: cb = 1
            sd2f += w*np.arccos(cb)**2

    return sd2 + sd2f

In [11]:
def thermo(params, stats, targets):
    """
    Calculates squared statistical distance loss function for configurational energies and forces.
    
    Parameters
    ----------
    params : list of lists and floats
             EAM interaction parameters (spline coefficients array and embedding function parameters)
    stats  : list of lists and floats
             Sufficient statistics
    targets: list of lists and floats
             target energies and forces
    
    Returns
    -------
    ge, he, se : floats
                 free energy, enthalpy, and entropy differences
    """

    for targ, stat in zip(targets, stats):
        beta = targ['beta'] # system inverse temperature
        u_targ = targ['energy'] # target energies
        u_stat = stat['energy'] # energy statistics

        # energy diference array for a given target trajectory
        uuu = beta*(utot_EAM(params, u_stat) - u_targ)
        uave = np.mean(uuu)
        uuu -= uave
        eee = np.exp(-uuu)

        ge = np.mean(eee)
        he = (np.mean(eee*(beta*hee + uuu + uave))/ge - beta*np.mean(hee))/beta

        ge = -np.log(ge)
        eee = np.exp(-0.5*(uuu - ge))
        ge = (ge + uave)/beta
        se = he - ge

    return ge, he, se

### 2.3 Target data

**Bulk properties of tungsten** (from _Marinica et.al_ except for B (bulk modulus) and Pc (Cauchy pressure)) T=0K?

| Property | BCC | FCC |
| :---     | :---: | :---: |
| a_0 (A) | 3.1648 | 4.054 |
| E_coh (eV/atom) | -8.9 | -8.43 |
| C11 (GPa) | 523 | - |
| C12 (GPa) | 203 | - |
| C44 (GPa) | 160 | - |
| B (GPa) | 310.4 | - |
| Pc (GPa) | 21.9 | - |

**Defect properties** (from _Marinica et.al_), based on 128+-1 atoms

Defect | Energy (eV)
:--- | :---:
(111) | 10.53
(110) | 10.82
(100) | 12.87
OCT | 13.11
TET | 12.27
Vacancy | 3.49

**List of available atomic configurations and DFT data**

1. Data from Marinica
  * 20 configurations generated from liquid Fe trajectory, N=113
  * DFT Forces
  * DFT Energy
2. Data from German (local minimum energy configs.)
  * Energy, Hessians?
  * Pure phases
    * BCC (54 atoms)
    * FCC (32 atoms)
  * Vacancy
    * vacancy (53 atoms)
  * Interstitial defects 
    * 110, 111 (55 atoms)
  * Screw dislocation
    * 111_easy_core (135 atoms)?

**Universal equation of state for metals at 0K**

In [13]:
# W parameters
l = 0.274
r_wse = 3.168
r_wse = 1.584
eta = 5.69
dE = 8.9

# equation of state. x is lattice expansion/compression parameter
def eos(x):
    a = (x - 1.0)*r_wse/l
    ene = np.exp(-a)
    ene *= -1.0 - a - 0.05*a**3
    return dE*ene

**Load target data files**

Structure of h5 files with configuration data.

1. Single target_conf file contains all the information from different simulations
2. Each simulation is treated as a trajectory (sometimes with only 1 configuration)
3. Trajectory data structure
    * overall description
        * General description
        * Particle id, element
    * per-configuration description
        * box description
        * particle coordinates ordered by particle number
        * configurational, kinetic, and total energies
        * forces per particle
        * 

### 2.4 Simulation details

## 3. Optimization

** Read pickled data**

In [38]:
import pickle

# load target data
with open(os.path.join(working, 'target'+'.pickle'), 'rb') as fi:
    targ_dict = pickle.load(fi)

# load stats data
with open(os.path.join(working, 'stats'+'.pickle'), 'rb') as fi:
    stats_dict = pickle.load(fi)
    
# load stats data
with open(os.path.join(working, 'pars_in'+'.pickle'), 'rb') as fi:
    pars_dict = pickle.load(fi)

In [39]:
targ_dict['dset1'].keys()

dict_keys(['type', 'weight', 'box', 'xyz', 'energy', 'temp', 'forces', 'beta'])

In [40]:
stats_dict['dset0']['energy']

[array([[  0.00000000e+00,   0.00000000e+00,   5.30470267e+01,
           1.75593619e+02,   2.97014262e+02,   5.10177394e+02],
        [  0.00000000e+00,   0.00000000e+00,   5.02878315e+01,
           6.03745127e+03,   4.94228670e+04,   4.30232846e+05],
        [  0.00000000e+00,   0.00000000e+00,   5.21108712e+01,
           5.70983685e+02,   1.63365688e+03,   4.82001802e+03]]),
 array([[  0.00000000e+00,   0.00000000e+00,   5.14550258e+01,
           1.70464399e+02,   2.88428719e+02,   4.95566130e+02],
        [  0.00000000e+00,   0.00000000e+00,   4.71552969e+01,
           5.67884370e+03,   4.65444808e+04,   4.05625877e+05],
        [  0.00000000e+00,   0.00000000e+00,   4.99675939e+01,
           5.48384801e+02,   1.56997576e+03,   4.63468054e+03]]),
 array([[  8.44107144e-01,   2.85999494e+00,   5.92911700e+01,
           1.85073450e+02,   3.09049456e+02,   5.27399971e+02],
        [  4.53796910e-03,   1.06712274e-01,   8.82656903e+01,
           7.15801700e+03,   5.50390964e+04,

In [32]:
pars_dict

{'embed': [-0.82756444522907735, 0.0, 2.2224833877071661e-05],
 'pair': [-0.17418264521969035,
  7.6813060863722065,
  1.6921565935750296,
  0.89780949157203727,
  -0.93943800806449307,
  0.18581577387791415]}

In [41]:
pars_in = [pars_dict['embed'][0], pars_dict['embed'][2], *pars_dict['pair']]
pars_in

[-0.82756444522907735,
 2.2224833877071661e-05,
 -0.17418264521969035,
 7.6813060863722065,
 1.6921565935750296,
 0.89780949157203727,
 -0.93943800806449307,
 0.18581577387791415]

In [50]:
stats = []
for k, v in stats_dict.items():
    print(v['energy'][0])
    dct ={'energy':valu, 'forces':valf}
    stats.append(dct)

[[  0.00000000e+00   0.00000000e+00   5.30470267e+01   1.75593619e+02
    2.97014262e+02   5.10177394e+02]
 [  0.00000000e+00   0.00000000e+00   5.02878315e+01   6.03745127e+03
    4.94228670e+04   4.30232846e+05]
 [  0.00000000e+00   0.00000000e+00   5.21108712e+01   5.70983685e+02
    1.63365688e+03   4.82001802e+03]]
[[  1.99433497e+01   4.42389319e+01   1.80667351e+02   4.18244472e+02
    6.44134609e+02   1.05542974e+03]
 [  4.08717808e-01   5.60422387e+00   9.90640840e+02   2.54535235e+04
    1.37679380e+05   9.56721872e+05]
 [  4.78124149e+00   2.04517147e+01   3.06169058e+02   1.60178615e+03
    3.76892775e+03   1.00467523e+04]]
[[  0.00000000e+00   5.90864237e-01   5.37197136e+01   1.76035288e+02
    2.97297010e+02   5.10355754e+02]
 [  0.00000000e+00   1.24774658e-05   5.32228812e+01   6.10468927e+03
    4.96349066e+04   4.30930912e+05]
 [  0.00000000e+00   1.19714373e-02   5.34971125e+01   5.73957884e+02
    1.63689840e+03   4.82356820e+03]]


In [ ]:
output = fmin(sd2_loss, pars_in, args=(stats, targets, weights), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
params_opt = output[0]
print(*params_opt)